In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import re
import nltk

#from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter#Counter计数器，统计字符出现的个数
from string import punctuation
import jieba
import jieba.analyse
import jieba.posseg as pseg
#import codecs, sys

In [2]:
#STOP_WORDS = nltk.corpus.stopwords.words()#英文中停用词

In [3]:
#with open('E:/维基百科中文20190720/extracted/AA/wiki_02_jian.txt','r',encoding='utf-8') as f:
#    data = pd.read_table(f)
#data.head(10)

数据预处理

In [4]:
# 英文标点符号+中文标点符号
punc = punctuation + u'.,;《》？！“”‘’@#￥%…&×（）——+【】{};；●，。&～、|\s::a-zA-Z0-9:「」'
def clean(line):
    # we will learn the regular expression next course.
    line_re = re.sub(r"[{}]+".format(punc),"",line)
    return line_re

繁转简

In [5]:
from hanziconv import HanziConv
def toSimple(line):
    return HanziConv.toSimplified(line)

分词

In [6]:
def stopwordslist(stop_path):
    stopwords = [line.strip() for line in open(stop_path,encoding='UTF-8-sig').readlines()]
    return stopwords

In [7]:
stop_path = './stop_word.txt'
STOP_WORDS = stopwordslist(stop_path)

In [8]:
#STOP_WORDS [1000:-1]

In [11]:
content = []
#target = codecs.open("E:/维基百科中文20190720/extracted/AA/wiki_02_jian_seg.txt", 'w', encoding="utf-8")   
with open('E:/维基百科中文20190720/extracted/AA/wiki_02_01.txt', "r", encoding='utf-8-sig') as f:
    for line in f.readlines():#一行一行读
        if len(line)<=1 or line.startswith("<doc") or line.startswith("</doc>"):continue
        line_simple = toSimple(line)
        line_clean = clean(line_simple)
        line_seg = " ".join(jieba.cut(line_clean))
        line_seg_w = line_seg.split(" ")
        for word in line_seg_w:
            if word in STOP_WORDS :
                line_seg_w.remove(word)          
        content.append(line_seg_w)
        
        #target.writelines(line_seg)
f.close()
#target.close()

In [12]:
content

[['贵州省', '遵义市', '第四', '中学'],
 ['贵州省',
  '遵义市',
  '第四',
  '中学',
  '创立',
  '年现',
  '位于',
  '中华人民共和国',
  '贵州省',
  '遵义市',
  '红花岗区',
  '石龙',
  '路',
  '石龙',
  '路',
  '校区',
  '新',
  '蒲',
  '校区',
  '两个',
  '校区',
  '贵州省',
  '一类',
  '省级',
  '示范',
  '高级中学'],
 ['新',
  '蒲',
  '校区',
  '占地',
  '亩',
  '含',
  '增修',
  '运动场',
  '亩',
  '校园',
  '面对',
  '环境优美',
  '湿地',
  '公园',
  '教学楼',
  '实验楼',
  '学生公寓',
  '图书',
  '科技',
  '楼',
  '艺术',
  '楼',
  '体育馆',
  '后勤',
  '服务中心',
  '对外',
  '交流中心',
  '等栋',
  '大楼',
  '总',
  '建筑面积',
  '万多平方米'],
 ['年级',
  '男女生',
  '单独',
  '四合院',
  '学生公寓',
  '每室',
  '或',
  '进行',
  '管理',
  '寝室',
  '配置',
  '高档',
  '实木',
  '床位',
  '每人',
  '配有',
  '书桌',
  '座椅',
  '立体',
  '衣柜',
  '每室',
  '配置',
  '有',
  '淋浴器',
  '饮水机',
  '卫生间',
  '阳台',
  '设施'],
 ['遵义',
  '四中',
  '十二',
  '放眼',
  '石',
  '雕刻',
  '世界',
  '世界',
  '知名',
  '大学',
  '学校',
  '精神',
  '引导',
  '学生',
  '放眼世界',
  '开阔',
  '胸襟',
  '情怀',
  '石',
  '告诫',
  '学生',
  '做人',
  '落到',
  '细节',
  '处',
  '培养',
  '百姓',
  '情怀',
  '引凤屋',
  '体现',
  '人与自然',
  '和谐'

词向量

In [14]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

In [35]:
#path = get_tmpfile("word2vec.model")

#model = Word2Vec(content, size=100, window=5, min_count=1, workers=4)
#model.save("word2vec_02.model")#通过model.save()方法保存的序列文件模型可重新训练，使用model.save_word2vec_format()保存的模型不能重新训练。

In [15]:
model = Word2Vec.load("word2vec_02.model")#加载

In [70]:
"""
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [72]:
#tsne_plot(model)

In [24]:
sentence = [["娟宝"]]
model.build_vocab(sentence,update=True)#添加语料库
model.train(sentence, total_examples=model.corpus_count, epochs=model.iter)#model.train()？？只可以设置增加新语料，设置迭代次

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(0, 5)

In [17]:
model.wv["遵义"]  # numpy vector of a word

array([-0.72789854, -0.01197063, -0.825266  , -1.0493054 ,  1.2379963 ,
        1.2541939 , -0.7163561 , -0.19542445, -1.3018558 , -0.06442703,
       -1.2371435 , -1.1490746 ,  0.5886354 ,  0.49263668, -0.3357093 ,
       -0.07709107, -0.1432251 , -0.2980808 , -0.6563876 ,  0.8555254 ,
        0.15089794,  1.2945104 , -1.1183175 , -1.5090988 , -0.9794747 ,
       -0.5310611 ,  0.35512996,  0.28437865,  0.64084786,  2.131619  ,
       -1.2583109 ,  0.05701883, -1.872722  , -1.1447929 , -0.41779304,
       -1.0264695 ,  0.16731949, -0.9842228 ,  0.06151897,  0.8281011 ,
        0.89745504,  0.38250118, -0.27745825, -1.1016595 ,  0.76795316,
        1.3368169 , -1.1496915 ,  0.5450534 , -0.6421246 ,  0.45349184,
       -2.360794  ,  0.27691075,  0.10910977, -0.24457201, -0.6700607 ,
       -0.28703892, -0.5365075 ,  0.7833475 ,  0.9841072 , -1.4270196 ,
       -0.2850986 ,  0.49068016, -1.3216846 ,  0.09335261,  0.37192887,
       -0.988116  , -0.23174007, -1.565447  , -0.9567968 ,  1.00

In [27]:
model.most_similar("牛王")#同类词

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('引凤', 0.9101300239562988),
 ('东龛', 0.9089977145195007),
 ('张连溪', 0.9065955877304077),
 ('风燿敬', 0.9061399102210999),
 ('优纪', 0.9047637581825256),
 ('双扉', 0.9044007658958435),
 ('神霄玉府', 0.9038353562355042),
 ('蟞', 0.9034807085990906),
 ('火出', 0.9028003215789795),
 ('张绮文', 0.9026411771774292)]